In [1]:
# importando as bibliotecas necessarias

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import numpy as np
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import pandas as pd
import cartopy
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from scipy.stats import norm
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
# from sklearn import datasets
from pandas.plotting import scatter_matrix
# import seaborn as sns

In [2]:
# lendo arquivo com dados terrestre para exportar as coordendas para o ICGEM

latlon_terrestre=pd.read_csv('dados/terrestre_input_icgem.txt', sep='\s+')
(latlon_terrestre)

FileNotFoundError: [Errno 2] File dados/terrestre_input_icgem.txt does not exist: 'dados/terrestre_input_icgem.txt'

In [ ]:
latlon_terrestre.to_csv('dados/latlon_terrestre.txt',sep='\t',header=True, columns=['Latitude', 'Longitude'])


In [ ]:
# lendo arquivo de altitude geométrica predita das estaçoes

header=['Longitude','Latitude','Geom']
h_predito = pd.read_csv('dados/output_icgem_altitude_geometrica_estacoes.dat',skiprows=38, sep='\s+',decimal=b'.',names=header, usecols=(1,2,4))
h_predito

In [ ]:
# lendo o arquivo de dados terrestres para incorporar ao data frame h predito
terrestre=pd.read_csv('dados/terrestre_porcessado.txt',sep='\s+')

# incorporando h predito ao dataframe 
terrestre['h_pred']=h_predito['Geom']
terrestre

#salvando um arquivo para testar os disturbios com h predito
terrestre.to_csv('dados/teste.txt',sep='\t',header=True)

In [ ]:
print(h_predito['Geom'].min())
print(h_predito['Geom'].max())
print(74.543)
print(718.83)

# Mapa de Altitude geometrica predita para o caminhamento

fig = plt.figure(figsize=(16,16))

fname='shape_bacia_do_parnaiba/bacia_parnaiba.shp' # importando o shape da bacia do parnaiba

ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

#feiçoes
#ax.add_feature(cfeature.COASTLINE)
#ax.add_feature(cfeature.BORDERS)
ax.set_extent([-50, -39.5, -3.5, -7.5], ccrs.PlateCarree())

#determinando limite dos estados
states = cfeature.NaturalEarthFeature(category='cultural',
                                      name='admin_1_states_provinces_shp',
                                      scale='50m',
                                      facecolor='none')

#adicionando a feicao dos estados
ax.add_feature(states, edgecolor='gray',linestyle=':', linewidth=2,alpha=0.4)

#adicionando a geometria do shape da bacia do parnaiba
ax.add_geometries(Reader(fname).geometries(),
                  ccrs.PlateCarree(),facecolor='none',edgecolor='black', linewidth=4,alpha=0.5)

#adicionando linhas de grade
g1 = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linestyle='--', linewidth=1,color='gray',alpha=0.8)

#adicionando o scatter com as informaçoes dos dataframes
f3=ax.scatter(terrestre.Longitude, terrestre.Latitude,
              c=terrestre.h_pred, 
              s=100,
              vmin=48,vmax=720,
              cmap='terrain', 
              transform=ccrs.PlateCarree())

#removendo os eixos do lado direito e de cima
g1.ylabels_right = False
g1.xlabels_top = False

#formatando os eixos para georreferenciar
g1.yformatter = LATITUDE_FORMATTER
g1.xformatter = LONGITUDE_FORMATTER

g1.xlabel_style = {'size': 14}
g1.ylabel_style = {'size': 14}
ax.set_title('Bacia do Parnaíba', fontsize=20, y=1.02)

v = np.linspace(48, 720, 7, endpoint=True)
cbar=plt.colorbar(f3, shrink=0.85,orientation='horizontal',pad=0.07,aspect=30,ticks=v)

cbar.set_label('$h_{\,predita}$ (m)',fontsize=22,labelpad=2)
cbar.ax.tick_params(labelsize=12,color='black',labelcolor='black')
plt.savefig('imagens/altitude geom predita campo.png',format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# calculo do residuos da altitude : Rh = hobs - hpred

Rh= terrestre.Elevation - terrestre.h_pred

# incorporando ao dataframe
terrestre['residuos_h'] = Rh
terrestre

In [ ]:
print(terrestre['residuos_h'].min())
print(terrestre['residuos_h'].max())


# Mapa de residuos da Altitude geometrica predita para o caminhamento
fig = plt.figure(figsize=(16,16))

fname='shape_bacia_do_parnaiba/bacia_parnaiba.shp' # importando o shape da bacia do parnaiba

ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

#feiçoes
#ax.add_feature(cfeature.COASTLINE)
#ax.add_feature(cfeature.BORDERS)
ax.set_extent([-50, -39.5, -3.5, -7.5], ccrs.PlateCarree())

#determinando limite dos estados
states = cfeature.NaturalEarthFeature(category='cultural',
                                      name='admin_1_states_provinces_shp',
                                      scale='50m',
                                      facecolor='none')

#adicionando a feicao dos estados
ax.add_feature(states, edgecolor='gray',linestyle=':', linewidth=2,alpha=0.4)

#adicionando a geometria do shape da bacia do parnaiba
ax.add_geometries(Reader(fname).geometries(),
                  ccrs.PlateCarree(),facecolor='none',edgecolor='black', linewidth=4,alpha=0.5)

#adicionando linhas de grade
g1 = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linestyle='--', linewidth=1,color='gray',alpha=0.8)

#adicionando o scatter com as informaçoes dos dataframes
f3=ax.scatter(terrestre.Longitude, terrestre.Latitude,
              c=terrestre.residuos_h,
              s=100,
              vmin=-62.5,vmax=62.5,
              cmap='terrain',
              transform=ccrs.PlateCarree())

#removendo os eixos do lado direito e de cima

g1.ylabels_right = False
g1.xlabels_top = False

#formatando os eixos para georreferenciar
g1.yformatter = LATITUDE_FORMATTER
g1.xformatter = LONGITUDE_FORMATTER

g1.xlabel_style = {'size': 14}
g1.ylabel_style = {'size': 14}
ax.set_title('Bacia do Parnaíba', fontsize=20, y=1.02)

v = np.linspace(-62.5, 62.5, 7, endpoint=True)
cbar=plt.colorbar(f3, shrink=0.85,orientation='horizontal',pad=0.07,aspect=30,ticks=v)

cbar.ax.tick_params(labelsize=12,color='black',labelcolor='black')
cbar.set_label('Residuos (m)',fontsize=18, labelpad=2)

plt.savefig('imagens/residuos altitude geom.png',format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:

ndat = len(Rh)
mean = Rh.mean()
std = Rh.std()

vmin = min(Rh)
vmax = max(Rh)
print(mean,std, vmin, vmax,ndat)
#h_predito.to_excel('dados/residuos_h.xlsx',header=True)

In [ ]:
text_mean = '%1.2f' % mean
text_std = '%1.2f' % std

print(text_mean, text_std)
type(text_mean)

In [ ]:
fig = plt.figure(figsize=(14,12))
plt.style.use('ggplot')

plt.hist(Rh, bins=100, density=True,color='green', alpha=0.77, rwidth=0.85)
plt.xlim(-30,60)
x = np.linspace(vmin,vmax, ndat,endpoint=True)
y = norm.pdf(x, np.mean(Rh), np.std(Rh))

plt.text(-20,0.04,'média='+text_mean,fontsize=15)
plt.text(-20,0.03,'desvio padrão='+text_std,fontsize=15)
plt.xlabel('Residuos (m)' , fontsize = 20, labelpad = 10)
plt.ylabel('Frequência (%)', fontsize = 20, labelpad = 10)
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.plot(x, y, '--r', linewidth=2)
plt.title('Histograma dos residuos ', fontsize=20, y=1.02)
plt.savefig('imagens/histograma_residuos_h.png',format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#dataframe de trabalho (para visualizar os canais)
terrestre

In [ ]:
#grafico de dispersao delta obs x delta pred

plt.figure(figsize=(14,12))

data = pd.DataFrame(terrestre, columns=['delta_obs','delta_pred'])
xd = terrestre.delta_obs
yd = terrestre.delta_pred

# sort the data
reorder = sorted(range(len(xd)), key = lambda ii: xd[ii])
xd = [xd[ii] for ii in reorder]
yd = [yd[ii] for ii in reorder]

# make the scatter plot
plt.scatter(xd, yd, alpha=0.4, marker='o', color='blue')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)

plt.text(20,0,'$R^2 = %0.2f$'% Rsqr, fontsize=15)
#plt.text(15,50,'f(x) = 0.9563x + 10.446', fontsize=15)
plt.xlabel("$\delta_{g\,\,observado}$ (mGal)", fontsize=22)
plt.ylabel("$\delta_{g\,\,predito}$ (mGal)",fontsize=22)
plt.title('$\delta_{g\,\,observado}$ X $\delta_{g\,\,predito}$',fontsize=22, y=1.02)
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

plt.plot(xl, yl, '-r')
plt.plot(xd, yerrLower, '--r')
plt.plot(xd, yerrUpper, '--r')
plt.savefig('imagens/dispersao delta obs x pred.png',format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#grafico de dispersao h obs x h pred

plt.figure(figsize=(14,12))

data = pd.DataFrame(terrestre, columns=['Elevation','h_pred'])
xd = terrestre.Elevation
yd = terrestre.h_pred

# sort the data
reorder = sorted(range(len(xd)), key = lambda ii: xd[ii])
xd = [xd[ii] for ii in reorder]
yd = [yd[ii] for ii in reorder]

# make the scatter plot
plt.scatter(xd, yd, alpha=0.4, marker='o', color='green')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)

plt.text(600,520,'$R^2 = %0.2f$'% Rsqr, fontsize=15)
#plt.text(15,50,'f(x) = 0.9563x + 10.446', fontsize=15)
plt.xlabel("$h_{\,observada}$ (m)", fontsize=22)
plt.ylabel("$h_{\,predita}$ (m)",fontsize=22)
plt.title('$h_{\,observada}$ X $h_{\,predita}$',fontsize=22,y=1.02)
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

plt.plot(xl, yl, '-r')
plt.plot(xd, yerrLower, '--r')
plt.plot(xd, yerrUpper, '--r')
plt.savefig('imagens/dispersao h obs x pred.png',format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#grafico de dispersao delta obs x h obs

plt.figure(figsize=(14,12))

data = pd.DataFrame(terrestre, columns=['Elevation','delta_obs'])
yd = terrestre.Elevation
xd = terrestre.delta_obs

# sort the data
reorder = sorted(range(len(xd)), key = lambda ii: xd[ii])
xd = [xd[ii] for ii in reorder]
yd = [yd[ii] for ii in reorder]

# make the scatter plot
plt.scatter(xd, yd, alpha=0.4, marker='o', color='purple')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)

plt.text(20,420,'$R^2 = %0.2f$'% Rsqr, fontsize=15)
#plt.text(15,50,'f(x) = 0.9563x + 10.446', fontsize=15)
plt.xlabel("$\delta_{g\,\,observado}$ (mGal)", fontsize=22)
plt.ylabel("$h_{\,\,observada}$ (m)",fontsize=22)
plt.title('$\delta_{g\,\,observado}$ X $h_{\,\,observada}$',fontsize=22, y=1.02)
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

plt.plot(xl, yl, '-r')
plt.plot(xd, yerrLower, '--r')
plt.plot(xd, yerrUpper, '--r')
plt.savefig('imagens/dispersao delta obs x h obs.png',format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#grafico de dispersao delta pred x h pred

plt.figure(figsize=(14,12))

data = pd.DataFrame(terrestre, columns=['h_pred','delta_pred'])
yd = terrestre.h_pred
xd = terrestre.delta_pred

# sort the data
reorder = sorted(range(len(xd)), key = lambda ii: xd[ii])
xd = [xd[ii] for ii in reorder]
yd = [yd[ii] for ii in reorder]

# make the scatter plot
plt.scatter(xd, yd, alpha=0.4, marker='o', color='orange')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)

plt.text(10,420,'$R^2 = %0.2f$'% Rsqr, fontsize=15)
#plt.text(15,50,'f(x) = 0.9563x + 10.446', fontsize=15)
plt.xlabel("$\delta_{g\,\,predito}$ (mGal)", fontsize=22)
plt.ylabel("$h_{\,\,predita}$ (m)",fontsize=22)
plt.title('$\delta_{g\,\,predito}$ X $h_{\,\,predita}$',fontsize=22, y=1.02)
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

plt.plot(xl, yl, '-r')
plt.plot(xd, yerrLower, '--r')
plt.plot(xd, yerrUpper, '--r')
plt.savefig('imagens/dispersao delta pred x h pred.png',format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#grafico de dispersao delta pred x h obs

plt.figure(figsize=(14,12))

data = pd.DataFrame(terrestre, columns=['Elevation','delta_pred'])
yd = terrestre.Elevation
xd = terrestre.delta_pred

# sort the data
reorder = sorted(range(len(xd)), key = lambda ii: xd[ii])
xd = [xd[ii] for ii in reorder]
yd = [yd[ii] for ii in reorder]

# make the scatter plot
plt.scatter(xd, yd, alpha=0.4, marker='o', color='orange')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)

plt.text(10,420,'$R^2 = %0.2f$'% Rsqr, fontsize=15)
#plt.text(15,50,'f(x) = 0.9563x + 10.446', fontsize=15)
plt.xlabel("$\delta_{g\,\,predito}$ (mGal)", fontsize=22)
plt.ylabel("$h_{\,\,observada}$ (m)",fontsize=22)
plt.title('$\delta_{g\,\,predito}$ X $h_{\,\,observada}$',fontsize=22, y=1.02)
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

plt.plot(xl, yl, '-r')
plt.plot(xd, yerrLower, '--r')
plt.plot(xd, yerrUpper, '--r')
plt.savefig('imagens/dispersao delta pred x h obs.png',format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#grafico de dispersao h obs x residuos delta

plt.figure(figsize=(14,12))

data = pd.DataFrame(terrestre, columns=['residuos','Elevation'])
xd = terrestre.residuos
yd = terrestre.Elevation

# sort the data
reorder = sorted(range(len(xd)), key = lambda ii: xd[ii])
xd = [xd[ii] for ii in reorder]
yd = [yd[ii] for ii in reorder]

# make the scatter plot
plt.scatter(xd, yd, alpha=0.4, marker='o', color='black')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)

plt.text(15,400,'$R^2 = %0.2f$'% Rsqr, fontsize=15)
#plt.text(15,50,'f(x) = 0.9563x + 10.446', fontsize=15)
plt.xlabel("Residuos (mGal)", fontsize=20)
plt.ylabel("$h_{\,\,observada}$ (m)",fontsize=20)
plt.title('Resíduos $\delta_{g}$ X $h_{\,\,observada}$',fontsize=22)

# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

plt.plot(xl, yl, '-r')
plt.plot(xd, yerrLower, '--r')
plt.plot(xd, yerrUpper, '--r')
plt.savefig('imagens/dispersao residuos delta x h obs.png',format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#grafico de dispersao h pred x residuos delta

plt.figure(figsize=(14,12))

data = pd.DataFrame(terrestre, columns=['residuos','h_pred'])
xd = terrestre.residuos
yd = terrestre.h_pred

# sort the data
reorder = sorted(range(len(xd)), key = lambda ii: xd[ii])
xd = [xd[ii] for ii in reorder]
yd = [yd[ii] for ii in reorder]

# make the scatter plot
plt.scatter(xd, yd, alpha=0.4, marker='o', color='blue')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)

plt.text(15,400,'$R^2 = %0.2f$'% Rsqr, fontsize=15)
#plt.text(15,50,'f(x) = 0.9563x + 10.446', fontsize=15)
plt.xlabel("Residuos (mGal)", fontsize=20)
plt.ylabel("$h_{\,\,predita}$ (m)",fontsize=20)
plt.title('Resíduos $\delta_{g}$ X $h_{\,\,predita}$',fontsize=22)

# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

plt.plot(xl, yl, '-r')
plt.plot(xd, yerrLower, '--r')
plt.plot(xd, yerrUpper, '--r')
plt.savefig('imagens/dispersao residuos delta x h pred.png',format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#grafico de dispersao residuos h x residuos delta

plt.figure(figsize=(14,12))

data = pd.DataFrame(terrestre, columns=['residuos','residuos_h'])
xd = terrestre.residuos
yd = terrestre.residuos_h

# sort the data
reorder = sorted(range(len(xd)), key = lambda ii: xd[ii])
xd = [xd[ii] for ii in reorder]
yd = [yd[ii] for ii in reorder]

# make the scatter plot
plt.scatter(xd, yd, alpha=0.4, marker='o', color='darkblue')

# determine best fit line
par = np.polyfit(xd, yd, 1, full=True)

slope=par[0][0]
intercept=par[0][1]
xl = [min(xd), max(xd)]
yl = [slope*xx + intercept  for xx in xl]

# coefficient of determination, plot text
variance = np.var(yd)
residuals = np.var([(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)])
Rsqr = np.round(1-residuals/variance, decimals=2)

plt.text(18,15,'$R^2 = %0.2f$'% Rsqr, fontsize=15)
#plt.text(15,50,'f(x) = 0.9563x + 10.446', fontsize=15)
plt.xlabel("Resíduos $\delta_{g}$ (mGal)", fontsize=20)
plt.ylabel("$Resíduos_{\,\,h}$ (m)",fontsize=20)
plt.title('Resíduos $\delta_{g}$ X $Resíduos_{\,\,h}$',fontsize=22)

# error bounds
yerr = [abs(slope*xx + intercept - yy)  for xx,yy in zip(xd,yd)]
par = np.polyfit(xd, yerr, 2, full=True)

yerrUpper = [(xx*slope+intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]
yerrLower = [(xx*slope+intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx,yy in zip(xd,yd)]

plt.plot(xl, yl, '-r')
plt.plot(xd, yerrLower, '--r')
plt.plot(xd, yerrUpper, '--r')
plt.savefig('imagens/dispersao residuos delta x residuos h.png',format='png', dpi=300, bbox_inches='tight')
plt.show()

## TODO: PESQUISAR A MATRIX PLOT DO PANDAS PARA FAZER O GRÁFICO DE DISPERSÃO ADEQUADAMENTE:
https://benalexkeen.com/scatter-matrices-using-pandas/ <br>
https://www.marsja.se/pandas-scatter-matrix-pair-plot/ <br>
https://pandas.pydata.org/docs/reference/api/pandas.plotting.scatter_matrix.html

In [ ]:
terrestre

In [ ]:
df = pd.DataFrame(terrestre, columns=['Elevation','delta_obs','delta_pred'])
df.rename(columns={'Elevation': '$h$'}, inplace = True)
#df.rename(columns={'delta': 'Resíduos ($\delta_{g}$)'}, inplace = True)
#df.rename(columns={'residuos_h': 'Resíduos ($h$)'}, inplace = True)
scatter_matrix(df, alpha=0.5,figsize=(14,12),grid=True, hist_kwds={'bins':50}, color='blue')
plt.savefig('imagens/histogramas e residuos.png',format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
scatter_matrix(df, diagonal='kde',alpha=0.5,figsize=(14,12), grid=True, color='blue')

In [ ]:
# Matriz de correlaçao

cormat = terrestre.corr(method='pearson')
cormat

In [ ]:
def triang(cormat, triang='lower'):
    
    if triang == 'upper':
        rstri = pd.DataFrame(np.triu(cormat.values),
                             index=cormat.index,
                             columns=cormat.columns).round(3)
        rstri = rstri.iloc[:,1:]
        rstri.drop(rstri.tail(1).index, inplace=True)
        
    if triang == 'lower':
        rstri = pd.DataFrame(np.tril(cormat.values),
                             index=cormat.index,
                             columns=cormat.columns).round(3)
        rstri = rstri.iloc[:,:-1]
        rstri.drop(rstri.head(1).index, inplace=True) 
 
    rstri.replace(to_replace=[0,1], value='', inplace=True)
    
    return(rstri)

In [ ]:
triang(cormat, 'upper')

In [ ]:
mask = np.zeros_like(cormat, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True


fig = plt.figure(figsize=(14, 12))
sns.heatmap(cormat, mask=mask, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .7},cmap="rainbow")
plt.savefig('imagens/matriz de correlaçao.png',format='png', dpi=300, bbox_inches='tight')